## Цель проекта:

 Создать приложение для генерации упражнений по английскому языку на основе различных текстов.
 
 
##  Задачи:
1. Изучить имеющиеся примеры упражнений
2. Разработать методы преобразования текста в подобные задания
3. Оформить в виде функций или классов и выделить в модуль
4. Создать приложение для демонстрации заказчику

## Импорт библиотек

In [2]:
import numpy as np
import pandas as pd
import random
from random import choice

import pyinflect
import spacy
import en_core_web_sm

from gensim.test.utils import lee_corpus_list
from gensim.models import Word2Vec
import gensim.downloader as api

from english_exercise import EnglishExercise

In [3]:
# малая модель spacy
nlp = spacy.load("en_core_web_sm")
# малая модель glove wiki
model = api.load("glove-wiki-gigaword-100") 

### Создание датафрейма и парсеровщика текста

In [4]:
data = pd.DataFrame(columns = ['raw', 'type', 'object', 'options', 'answer'])

with open('Little.txt') as file:
    for line in file:
        line = line.strip()   # обрезаем лишние отступы
        if len(line) > 0:  # проверяем не нулевая ли длинна
            doc = nlp(line) 
            for sent in doc.sents:
                data.loc[len(data), 'raw'] = sent.text

In [12]:
data

,raw,type,object,options,answer
0,Little Red Cap,NaN,NaN,NaN,NaN
1,Jacob and Wilhelm Grimm,NaN,NaN,NaN,NaN
2,Once upon a time there was a sweet little girl.,NaN,NaN,NaN,NaN
3,"Everyone who saw her liked her, but most of al...",NaN,NaN,NaN,NaN
4,Once she gave her a little cap made of red vel...,NaN,NaN,NaN,NaN
...,...,...,...,...,...
108,The German title of this tale is RotkГ¤ppchen,NaN,NaN,NaN,NaN
109,(RothkГ¤ppchen in the nineteenth-century spell...,NaN,NaN,NaN,NaN
110,Link to an English translation of the Grimms' ...,NaN,NaN,NaN,NaN
111,Link to the German text of the Grimms' final v...,NaN,NaN,NaN,NaN


### Создание класса из 4 функций

In [5]:
class EnglishExercise():
   
    def choose_variant(self, row):
        random_var = random.choice([self.choose_verb_form, self.fill_the_gap, 
                                    self.choose_the_sent, self.select_word])
        row['type'] = random_var.__name__
    
        if row['type'] == 'choose_verb_form':
            return self.choose_verb_form(row)
        elif row['type'] == 'fill_the_gap':
            return self.fill_the_gap(row)
        elif row['type'] == 'choose_the_sent':
            return self.choose_the_sent(row)
        elif row['type'] == 'select_word':
            return self.select_word(row)
        else:
            return row

    
    
    def fill_the_gap(self, row):
        if row['type'] == 'fill_the_gap':
            
            token_list = []
            for token in nlp(row['raw']):
                if token.pos_ in ['ADJ', 'NOUN', 'VERB']:
                    token_list.append(token.text)
            if len(token_list) == 0:
                return row
            token_random = random.choice(token_list)
            row['options'] = []
            row['object'] = token_random
            row['answer'] = token_random
            row['description'] = 'Заполните пропуск'
            return row
    
    
    def choose_verb_form (self, row):
        token_list = []
        if row['type'] == 'choose_verb_form': 
            for token in nlp(row['raw']):
                if token.pos_ == 'VERB':
                    token_list.append(token)
        if len(token_list) == 0:
            return row
        token_random = random.choice(token_list)
        try:
            options = []
            if token_random.text == token_random._.inflect('VBG'):
                options.extend([token_random.text, token_random._.inflect('VBD'), token_random._.inflect('VBP')])
            elif token_random.text == token_random._.inflect('VBD'):
                options.extend([token_random.text, token_random._.inflect('VBG'), token_random._.inflect('VBP')])
            else:
                options.extend([token_random._.inflect('VBG'), token_random._.inflect('VBD'), token_random.text])
        
            random.shuffle(options)
            row['options'] = options
            row['type'] = 'choose_verb_form'
            row['answer'] = token_random.text
            row['description'] = 'Выберите правильный глагол'
            row['object'] = token_random.text
        except:
               pass
     
        return row  
    
    
    def choose_the_sent(self, row):
        token_list = []
        if row['type'] == 'choose_the_sent':
            words = row['raw'].split()
            if len(words) < 4 or len(words) > 10:
                return row
    
            new_sent_1, new_sent_2 = row['raw'], row['raw']
    
           
            for token in nlp(row['raw']):
                if token.pos_ in ['NOUN', 'VERB', 'ADJ']:
                    token_list.append(token)
            if len(token_list) == 0:
                return row           
                
            for token in token_list:
                try:

                    new_word_1 = model.most_similar(token.text.lower())[1][0]
                    new_word_2 = model.most_similar(positive = [token.text.lower(), 'bad'],
                                        negative = ['good'],
                                        )[2][0]

                    new_word_1 = new_word_1.title() if token.text.istitle() else new_word_1
                    new_word_2 = new_word_2.title() if token.text.istitle() else new_word_2
        
                    new_sent_1 = new_sent_1.replace(token.text, new_word_1)
                    new_sent_2 = new_sent_2.replace(token.text, new_word_2)
                except:
                    pass               
                    
                
                options = [new_sent_1, new_sent_2, row['raw']]
                row['type'] = 'choose the sent'
                row['options'] = options
                row['answer'] = row['raw']
                row['description'] = 'Выберите правильное предложение'      
                return row       
    
    def select_word (self, row):

        token_list = []
        if row['type'] == 'select_word':
            for i in nlp(row['raw']):
                if i.pos_ in ['NOUN', 'VERB', 'ADJ']:
                    token_list.append(i)
            if len(token_list) == 0:
                return row
            token_random = random.choice(token_list)
            try:
                options = [w[0] for w in model.similar_by_word(token_random.text.lower(), topn = 2)] + [token_random.text] 
        
                if token_random.text.istitle():
                    options = [x.title() for x in options]
                random.shuffle(options) 
                
                row['options'] = options 
                row['object'] = token_random.text
                row['answer']  = token_random.text
                row['description'] = 'Выберите наиболее подходящее слово'
            except:
                  pass
            return row   


### Применение класса к таблице с текстом

In [9]:
exercise = EnglishExercise()

new_data = data.apply(exercise.choose_variant, axis=1)

In [10]:

new_data 

,answer,description,object,options,raw,type
0,Little,NaN,Little,[],Little Red Cap,choose_verb_form
1,NaN,NaN,NaN,NaN,Jacob and Wilhelm Grimm,choose_verb_form
2,was,Выберите правильный глагол,was,"[was, am, being]",Once upon a time there was a sweet little girl.,choose_verb_form
3,child,Выберите наиболее подходящее слово,child,"[child, children, mother]","Everyone who saw her liked her, but most of al...",select_word
4,red,Выберите наиболее подходящее слово,red,"[blue, yellow, red]",Once she gave her a little cap made of red vel...,select_word
...,...,...,...,...,...,...
108,German,NaN,German,[],The German title of this tale is RotkГ¤ppchen,select_word
109,century,NaN,century,[],(RothkГ¤ppchen in the nineteenth-century spell...,choose_verb_form
110,Link,Выберите правильный глагол,Link,"[Linking, Link, Linked]",Link to an English translation of the Grimms' ...,choose_verb_form
111,die,Выберите правильный глагол,die,"[died, dying, die]",Link to the German text of the Grimms' final v...,choose_verb_form


In [8]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113 entries, 0 to 112
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   answer       95 non-null     object
 1   description  95 non-null     object
 2   object       82 non-null     object
 3   options      95 non-null     object
 4   raw          113 non-null    object
 5   type         113 non-null    object
dtypes: object(6)
memory usage: 10.2+ KB


## Выводы:

В ходе работы были использованы модели nlp:  малая модель spacy и малая модель glove wiki.

Был разработан универсальный парсеровщик текста, текст преобразован в dataframe.

Был создан класс, состоящий из 4рех методов. Методы представляют собой следующие упражнения по английскому языку:


- Упражнение 1 - fill_the_gap (заполните пропуски). Метод принимает на вход предложение из текста. Затем модель разбивает предложение на токены, ищет в нем существительные, прилагательные и глаголы. Из этих частей речи рандомно выбирает слово, которое заменет на пропуск. Пользователю необходимо по контексту догадаться какое слово пропущенно и вписать его. Упражнение на лексику.

- Упражнение 2 - choose_verb_form (выберите правильную форму глагола). Принимает на вход строку из текста и ищет в ней только гаголы. Пользователю предлагается 3 варианта формы глагола выбор. Это грамматическое упражнение, которое тренирует грамматику, времена.

- Упражнение 3 -  choose_the_sent (Выберите правильное предложение). Таже работает с отдельной строкой. Модель ищет в предложении существительные, прилагательные, глаголы. Рандомно выбрав несколько токенов, она подменяет их на близкие по значению, пользуясь методом most_similar. Генерируются похожие предложения с подменой нескольких слов. Пользователю предлагается 3 варианта предложения на выбор. Упражнение на понимание смысла текста и лексику.


- Упражнение 4 - select_word (выберте правильное слово). Работает с отдельной строкой и ищет в ней существительные, прилагательные и глаголы. ПРедлагает пользователю несколько синонимов, из которых необходимо выбрать наиболее подходящий.


Также реализован демонстрационный вариант приложения в streamlit.